In [1]:
import os
import numpy as np
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# from fastvit import fastvit_t8
# keras_model = fastvit_t8()
# keras_model.summary(expand_nested=True)
# keras_model.load_weights("fastvit_t8_patch_embed.weights.h5", skip_mismatch=False)
# # for i in keras_model.variables:
# #     print(i.path, i.value.sum())

In [3]:
# from pt_fastvit import fastvit_t8 as pt_fastvit_t8
# from torchinfo import summary

# pt_model = pt_fastvit_t8(inference_mode=False) 

# pt_model.load_state_dict(torch.load("pt_fastvit_t8_patch_embed.pth", weights_only=True), strict=False)
# # for k, v in pt_model.state_dict().items():
# #     print(k, v.sum())

# summary(pt_model, (1, 3, 256, 256))

In [4]:
import os
import numpy as np
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from pt_fastvit import fastvit_t8 as pt_fastvit_t8
from pt_fastvit import fastvit_t12 as pt_fastvit_t12
from pt_fastvit import fastvit_ma36 as pt_fastvit_ma36
from torchinfo import summary

pt_model = pt_fastvit_ma36(inference_mode=False) 

pt_model.load_state_dict(torch.load("fastvit_ma36.pth.tar", weights_only=True)["state_dict"], strict=True)
# pt_model.load_state_dict(torch.load("pt_fastvit_t8_patch_embed.pth", weights_only=True), strict=False)
# for k, v in pt_model.state_dict().items():
#     print(k, v.sum())

# summary(pt_model, (1, 3, 256, 256))
# pt_model

c:\Users\vaibh\miniconda3\envs\keras_univ\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


<All keys matched successfully>

In [7]:
# ImageNet normalization constants for PyTorch (tensors)
pt_IMAGENET_DEFAULT_MEAN = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
pt_IMAGENET_DEFAULT_STD = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)

activations = {}

def capture_output(module, input, output):
    # print(str(module))
    activations["input"] = input[0].detach()
    activations['output'] = output.detach()


pt_model.conv_exp.register_forward_hook(capture_output)

def test_prediction_pytorch(*, image_path, model, image_shape=(224, 224), show=False):
    # Load the image using OpenCV
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, image_shape, interpolation=cv2.INTER_CUBIC)  # Resize to desired shape

    if show:
        plt.imshow(img)
        plt.axis("off")
        plt.show()

    # Normalize pixel values to [0, 1]
    img = img.astype(np.float32) / 255.0

    # Convert the image to a PyTorch tensor and rearrange dimensions: HWC -> CHW
    img_tensor = torch.from_numpy(img).permute(2, 0, 1)

    # Apply ImageNet normalization: (input - mean) / std
    # The mean and std tensors are reshaped to broadcast across height and width
    img_tensor = (img_tensor - pt_IMAGENET_DEFAULT_MEAN[:, None, None]) / pt_IMAGENET_DEFAULT_STD[:, None, None]

    # Add a batch dimension: shape becomes (1, C, H, W)
    img_tensor = img_tensor.unsqueeze(0)
    print(img_tensor.sum(), img_tensor.shape)

    # Ensure the model is in evaluation mode and perform inference without gradients
    model.eval()
    with torch.no_grad():
        preds = model(img_tensor)

    print("preds.shape", preds.shape)
    print("preds.sum()", preds.sum().item())
    print("preds.sum()", preds.argmax(dim=1).item())

    # # Get the index of the highest score (predicted class)
    predicted_class = preds.argmax(dim=1).item()
    # print(f"Model: {model.__class__.__name__}, Predictions: {predicted_class}")


test_prediction_pytorch(image_path=r"C:\Users\vaibh\OneDrive\Desktop\Work\other_work_mine\Mine\keras-vision\test_images\pandas.JPG", model=pt_model, image_shape=(256, 256))


# print("--------")
# inputx = activations['input']
# print("input:", inputx.shape, inputx.sum())

print("--------")
# Retrieve the output from the SiLU activation layer
output = activations['output']
print("Output:", output.shape, output.sum())

# print(logits.argmax())

tensor(-59884.4219) torch.Size([1, 3, 256, 256])
preds.shape torch.Size([1, 1000])
preds.sum() -2381.94921875
preds.sum() 388
--------
Output: torch.Size([1, 1216, 8, 8]) tensor(6865.1826)
